In [7]:
!pip install git+https://gitee.com/mindspore-lab/mindnlp.git@master

  Cloning https://gitee.com/mindspore-lab/mindnlp.git (to revision master) to /tmp/pip-req-build-oy4yu1b5
  Running command git clone --filter=blob:none --quiet https://gitee.com/mindspore-lab/mindnlp.git /tmp/pip-req-build-oy4yu1b5
  Resolved https://gitee.com/mindspore-lab/mindnlp.git to commit 95f0253a0f03091b2cf88328d1bbd5de10b8aebf
  Preparing metadata (setup.py) ... done
  Created wheel for mindnlp: filename=mindnlp-0.4.1-py3-none-any.whl size=8702379 sha256=e31709b04ef894a6142703b68082c6e8bed61655d153d437a0685a92c9d31230
  Stored in directory: /tmp/pip-ephem-wheel-cache-391eksf7/wheels/48/6f/01/481d9cc3fc1b06959e0d577c06b7f1e45664f21d0cd09cb425
Successfully built mindnlp
  Attempting uninstall: mindnlp
    Found existing installation: mindnlp 0.4.0
    Uninstalling mindnlp-0.4.0:
      Successfully uninstalled mindnlp-0.4.0


In [1]:
pip show mindnlp

Name: mindnlp
Version: 0.4.1
Summary: An open source natural language processing research tool box. Git version: [sha1]:95f0253a, [branch]: (HEAD -> master, origin/master, origin/HEAD)
Home-page: https://github.com/mindlab-ai/mindnlp/tree/master/
Author: MindSpore Team
Author-email: 
License: Apache 2.0
Location: /home/snape/miniconda3/envs/mindspore_py39/lib/python3.9/site-packages
Requires: addict, datasets, evaluate, mindspore, ml-dtypes, pillow, pyctcdecode, pytest, regex, requests, safetensors, sentencepiece, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [13]:
import gradio as gr
import mindspore
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.peft import PeftModel, IA3Config

In [7]:
import os

config = IA3Config.from_pretrained("/home/snape/roberta-large_IA3_SEQ_CLS")
model = AutoModelForSequenceClassification.from_pretrained("/home/snape/", local_files_only=True)
model = PeftModel.from_pretrained(model, "/home/snape/roberta-large_IA3_SEQ_CLS")
tokenizer = AutoTokenizer.from_pretrained("AI-ModelScope/roberta-large", padding_side="right", mirror='modelscope')

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /home/snape/ and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 878k/878k [00:00<00:00, 2.46MB/s]
100%|██████████| 446k/446k [00:00<00:00, 1.67MB/s]
100%|██████████| 1.29M/1.29M [00:00<00:00, 2.15MB/s]
/home/snape/miniconda3/envs/mindspore_py39/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [12]:
from mindspore import Tensor

def predict(sentence1, sentence2):
    inputs = tokenizer(sentence1, sentence2, truncation=True, padding=True, max_length=128, return_tensors="ms")
    outputs = model(Tensor(inputs['input_ids']), Tensor(inputs['attention_mask']))
    logits = outputs.logits
    prediction = logits.argmax(axis=-1).asnumpy()[0]

    if prediction == 1:
        return "同义"
    else:
        return "不同义"

In [11]:
app = gr.Interface(fn=predict,
                     inputs=["text", "text"],
                     outputs="text",
                     title="文本分类模型",
                     description="请输入两个句子进行分类，判断它们是否同义")

app.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://2801667f72a8905226.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
